In [1]:
import pandas as pd, numpy as np, json

In [4]:
from utils import party_color_links,\
    get_link_color,\
    party_normalizer,\
    party_normalizer2,\
    country_normalizer,\
    hu_country,\
    get_photo,get_photos,get_url,\
    load_file, save_local, load_local

In [5]:
eu_vt=load_local('eu_vt')

In [6]:
eu_allegiance=load_local('eu_allegiance')

In [7]:
voteid_2_dossierid=load_local('voteid_2_dossierid')

In [8]:
countries=sorted(eu_vt)

Allegiance

In [9]:
def get_allegiance_matrix(key,vt,allegiance):
    allegiance_matrix={}
    initvote={'Same':0,'Opposite':0,'Total':0}
    for j1 in vt:
        outcome=j1['outcome']
        name1=j1[key]
        if name1 not in allegiance_matrix:allegiance_matrix[name1]={}
        if outcome=='For':
            for name2 in allegiance[j1['vote']]['For']:
                if name2 not in allegiance_matrix[name1]:
                    allegiance_matrix[name1][name2]=dict(initvote)
                allegiance_matrix[name1][name2]['Total']+=1
                allegiance_matrix[name1][name2]['Same']+=1
            if 'Against' in allegiance[j1['vote']]:
                for name2 in allegiance[j1['vote']]['Against']:
                    if name2 not in allegiance_matrix[name1]:
                        allegiance_matrix[name1][name2]=dict(initvote)
                    allegiance_matrix[name1][name2]['Total']+=1
                    allegiance_matrix[name1][name2]['Opposite']+=1
        elif outcome=='Against':
            for name2 in allegiance[j1['vote']]['Against']:
                if name2 not in allegiance_matrix[name1]:
                    allegiance_matrix[name1][name2]=dict(initvote)
                allegiance_matrix[name1][name2]['Total']+=1
                allegiance_matrix[name1][name2]['Same']+=1
            if 'For' in allegiance[j1['vote']]:
                for name2 in allegiance[j1['vote']]['For']:
                    if name2 not in allegiance_matrix[name1]:
                        allegiance_matrix[name1][name2]=dict(initvote)
                    allegiance_matrix[name1][name2]['Total']+=1
                    allegiance_matrix[name1][name2]['Opposite']+=1
    for j in allegiance_matrix:
        for i in allegiance_matrix[j]:
            allegiance_matrix[j][i]['Same_perc']=np.round(allegiance_matrix[j][i]['Same']/allegiance_matrix[j][i]['Total'],3)
            allegiance_matrix[j][i]['Opposite_perc']=np.round(allegiance_matrix[j][i]['Opposite']/allegiance_matrix[j][i]['Total'],3)
    return allegiance_matrix

In [18]:
init_areas=[]
override_areas=True
init_topics=[]
override_topics=False

In [19]:
eu_allegiance_matrix={}
for country in countries:
    for year in sorted(eu_vt[country]):
        print(country,year)
        votes_area_topic={}
        for vote in eu_vt[country][year]:
            if not override_topics:
                topics=get_vote_topics(vote['vote'])
            else:
                topics=list(init_topics)
            topics.append('All topics')    
            if not override_areas:
                areas=get_vote_areas(vote['vote'])
            else:
                areas=list(init_areas)
            areas.append('All areas')    
            for area in areas:
                for topic in topics:
                    if area not in votes_area_topic:votes_area_topic[area]={}
                    if topic not in votes_area_topic[area]:votes_area_topic[area][topic]=[]
                    votes_area_topic[area][topic].append(vote)
        for area in sorted(votes_area_topic):
            for topic in sorted(votes_area_topic[area]):    
                for allegiance_type1 in ['name','group','party']:
                    for allegiance_type2 in ['name','group','party']:
                        dummy=get_allegiance_matrix(allegiance_type1,votes_area_topic[area][topic],
                                eu_allegiance[country][year][allegiance_type2])
                        if dummy!={}:
                            if country not in eu_allegiance_matrix:
                                eu_allegiance_matrix[country]={}
                            if year not in eu_allegiance_matrix[country]:
                                eu_allegiance_matrix[country][year]={}
                            if area not in eu_allegiance_matrix[country][year]:
                                eu_allegiance_matrix[country][year][area]={}
                            if topic not in eu_allegiance_matrix[country][year][area]:
                                eu_allegiance_matrix[country][year][area][topic]={}
                            if allegiance_type1 not in eu_allegiance_matrix[country][year][area][topic]:
                                eu_allegiance_matrix[country][year][area][topic][allegiance_type1]={}
                            if allegiance_type2 not in eu_allegiance_matrix[country][year][area][topic][allegiance_type1]:
                                eu_allegiance_matrix[country][year][area][topic][allegiance_type1][allegiance_type2]={}
                            eu_allegiance_matrix[country][year][area][topic][allegiance_type1][allegiance_type2]=dummy

France 2008
France 2013
France 2018
Germany 2008
Germany 2013
Germany 2018
Hungary 2008
Hungary 2013
Hungary 2018
Italy 2008
Italy 2013
Italy 2018
Joint 2008
Joint 2013
Joint 2018
United Kingdom 2008
United Kingdom 2013
United Kingdom 2018


In [20]:
keys=['Same_perc']
eu_allegiance_list=[]
clean_names={'name':set(),'group':set(),'party':set()}
for country in sorted(eu_allegiance_matrix):
    for year in sorted(eu_allegiance_matrix[country]):
        print(country,year)
        for area in sorted(eu_allegiance_matrix[country][year]):
            for topic in sorted(eu_allegiance_matrix[country][year][area]): 
                for allegiance_type1 in sorted(eu_allegiance_matrix[country][year][area][topic]):
                    for allegiance_type2 in sorted(eu_allegiance_matrix[country][year][area][topic][allegiance_type1]):
                        for name1 in sorted(eu_allegiance_matrix[country][year][area][topic][allegiance_type1][allegiance_type2]):
                            for name2 in sorted(eu_allegiance_matrix[country][year][area][topic][allegiance_type1][allegiance_type2][name1]):
                                if name1!=name2:
                                    clean_names[allegiance_type1].add(name1)
                                    clean_names[allegiance_type2].add(name2)
                                    dummy={'country':country,
                                            'year':year,
                                            'allegiance_type1':allegiance_type1,
                                            'allegiance_type2':allegiance_type2,
                                            'area':area,
                                            'topic':topic,
                                            'name1':name1,
                                            'name2':name2}
                                    for key in keys:
                                        dummy[key]=eu_allegiance_matrix[country][year][area][topic]\
                                            [allegiance_type1][allegiance_type2][name1][name2][key]
                                    eu_allegiance_list.append(dummy)

France 2008
France 2013
France 2018
Germany 2008
Germany 2013
Germany 2018
Hungary 2008
Hungary 2013
Hungary 2018
Italy 2008
Italy 2013
Italy 2018
Joint 2008
Joint 2013
Joint 2018
United Kingdom 2008
United Kingdom 2013
United Kingdom 2018


In [21]:
for i in clean_names:
    clean_names[i]=sorted(list(clean_names[i]))

In [24]:
open('ep/export/json/clean_names.json','w').write(json.dumps(clean_names))

18370

In [41]:
datarepo='C:/Users/csala/OneDrive - Lancaster University/Datarepo/ep/'

In [38]:
#!!! lots of memory required ~2GB per country
open(datarepo+'eu_allegiance_list.json','w').write(json.dumps(eu_allegiance_list))

4250113652

In [39]:
del eu_allegiance_list

In [42]:
zf = zipfile.ZipFile(datarepo+'eu_allegiance_list.zip', mode='w')
zf.write(datarepo+'eu_allegiance_list.json','data.json',compress_type=compression)
zf.close()

In [43]:
save_local(eu_allegiance_matrix,'allegiance_matrix')

Clusterings

In [26]:
groups=load_local('groups')
parties=load_local('parties')

In [27]:
names=load_local('names')

In [28]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [29]:
def dict_2_matrix(matrix,key,party_labels=False):
    labels=sorted(matrix)
    slabels=[]
    for i in range(len(labels)):
        label=labels[i]
        if label in names:
            if party_labels:
                party=party_normalizer(names[label]['Constituencies'][0]['party'])
                group=party_normalizer(names[label]['Groups'][0]['groupid'])
                slabels.append(str(label)+u' | '+str(party)+' | '+str(group))
            else:
                slabels.append(label)
        else:
            slabels.append(label)

    #extend to square matrix
    inner_keys=matrix[sorted(matrix)[0]]
    inner_keys=sorted(inner_keys[sorted(inner_keys)[0]])
    for name1 in  labels:
        for name2 in labels:
            if name2 not in matrix[name1]:
                matrix[name1][name2]={i:0 for i in inner_keys}
    return np.array([[matrix[name1][name2][key] for name2 in sorted(matrix[name1])] for name1 in labels]),slabels

In [30]:
def hier_cluster(matrix,level,th=1,key='Same_perc',party_labels=False,method='single', metric='euclidean',criterion='distance'):
    X,labelList=dict_2_matrix(matrix[level][level],key,party_labels)
    if len(X)>1:
        try:
            linked = linkage(X, method=method,metric=metric)
        except:
            try:
                linked = linkage(X, method=method,metric='euclidean')
            except:
                linked = linkage(X, method='single',metric='euclidean')
        f=fcluster(linked, th, criterion)
        labelList=[labelList[i]+' | '+str(f[i]) for i in range(len(labelList))]
        return linked,labelList
    else:
        return [],[]

Exctract clusters

In [31]:
def get_unique_parent_node(nodes_children,node):
    if node in leafs:
        return node
    elif len(nodes_children[node])>1:
        return node
    else:
        return get_unique_parent_node(nodes_children,nodes_children[node][0])
def get_unique_parent(node,node_dict,unique_node_set,root):
    if node not in node_dict: 
        return root
    elif node_dict[node] in unique_node_set:
        return node_dict[node]
    else:
        return get_unique_parent(node_dict[node],node_dict,unique_node_set,root)

Save for VEGA

In [32]:
def country_normalizer(country):
    sign='🇪🇺'
    if country=='Hungary': sign='🇭🇺'
    elif country=='Germany': sign='🇩🇪'
    elif country=='Belgium': sign='🇧🇪'
    elif country=='France': sign='🇫🇷'
    elif country=='Poland': sign='🇵🇱'
    elif country=='Italy': sign='🇮🇹'
    elif country=='United Kingdom': sign='🇬🇧'
    return country+' '+sign

In [34]:
master_tree={}
nc_levels=10
key='Same_perc'
mpruned_nodes=[]
for country in countries:
    for year in eu_allegiance_matrix[country]:
        print(country,year)
        for area in eu_allegiance_matrix[country][year]:
            for topic in eu_allegiance_matrix[country][year][area]:
                for allegiance in eu_allegiance_matrix[country][year][area][topic]:
                    uid=country+year+allegiance+area+topic
                    cluster_list=[]
                    clusterdummy={}
                    if country=='Joint':
                        nc_max=nc_levels*1+2
                    else:
                        nc_max=nc_levels*1
                    for nc in range(2,nc_max):
                        hc,hlabels=hier_cluster(eu_allegiance_matrix[country][year][area][topic],
                                                allegiance,nc,key,True,'complete','seuclidean','maxclust')
                        for i in hlabels:
                            hi=i.split('|')
                            name=hi[0].strip()
                            cluster_no=hi[-1].strip()
                            if name not in clusterdummy:
                                clusterdummy[name]={}
                                clusterdummy[name]['name']=name
                                clusterdummy[name]['cluster_level_'+str(nc_max)]=name
                                clusterdummy[name]['country']=country
                                clusterdummy[name]['cluster_level_1']=country
                            clusterdummy[name]['cluster_level_'+str(nc)]='c'+str(nc)+str(cluster_no)
                    cluster_list=list(clusterdummy.values())
                    #construct tree
                    leafs=sorted(clusterdummy)
                    nodes=[{'name':country}]
                    nodes_done=set()
                    nodes_children={}
                    for i in cluster_list:
                        for cluster_level in range(2,nc_max+1):
                            node=i['cluster_level_'+str(cluster_level)]
                            parent=i['cluster_level_'+str(cluster_level-1)]
                            if node not in nodes_done:
                                dummy={}
                                nodes_done.add(node)
                                dummy['name']=node
                                dummy['parent']=parent
                                if parent not in nodes_children:nodes_children[parent]=[]
                                nodes_children[parent].append(node)
                                nodes.append(dummy)
                    #get unique nodes
                    node_dict={i['name']:i['parent'] for i in nodes[1:]}
                    unique_nodes={}
                    for node in nodes_children:
                        unique_nodes[node]=get_unique_parent_node(nodes_children,node)
                    unique_node_set=set(unique_nodes.values()).union(set(leafs))
                    #prune
                    pruned_nodes=[]
                    for i in nodes:
                        dummy=i
                        name=i['name']
                        if 'parent' not in i:
                            pruned_nodes.append(i)
                        elif i['name'] in unique_node_set:
                            dummy['parent']=get_unique_parent(name,node_dict,unique_node_set,nodes[0]['name'])
                            if name in leafs:
                                if allegiance=='name':
                                    party=names[name]['Constituencies'][0]['party']
                                    group=names[name]['Groups'][0]['groupid']
                                    home_country=names[name]['Constituencies'][0]['country']
                                    if type(party)==list:
                                        party=party[0]
                                    if type(group)==list:
                                        group=group[0]
                                    if type(home_country)==list:
                                        home_country=home_country[0]
                                    dummy['party']=party_normalizer(party)
                                    dummy['group']=party_normalizer(group)
                                    dummy['party2']=party_normalizer2(party)
                                    dummy['group2']=party_normalizer2(group)
                                else:
                                    dummy['party']=''
                                    dummy['group']=''
                                    dummy['party2']=''
                                    dummy['group2']=''
                                dummy['image']=get_photo(name,names,allegiance)     
                                dummy['url']=get_url(name,names,allegiance)
                            pruned_nodes.append(dummy)
                    for i in pruned_nodes:
                        dummy=i
                        if 'party' in dummy:
                            dummy['partycolor']=get_link_color(dummy['party'])
                        if 'group' in dummy:
                            dummy['groupcolor']=get_link_color(dummy['group'])
                        dummy['country']=country
                        dummy['home_country']=home_country
                        dummy['home_country2']=country_normalizer(home_country)
                        dummy['year']=year
                        dummy['area']=area
                        dummy['topic']=topic
                        dummy['allegiance']=allegiance
                        mpruned_nodes.append(dummy)

France 2008
France 2013
France 2018
Germany 2008
Germany 2013
Germany 2018
Hungary 2008
Hungary 2013
Hungary 2018
Italy 2008
Italy 2013
Italy 2018
Joint 2008
Joint 2013
Joint 2018
United Kingdom 2008
United Kingdom 2013
United Kingdom 2018


In [35]:
open('ep/export/json/topics.json','w').write(json.dumps(mpruned_nodes))

44872418

In [36]:
save_local(mpruned_nodes,'topics')

In [37]:
del mpruned_nodes